In [16]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [17]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [18]:
!pip install pyserial

In [5]:
'''import serial.tools '''
import serial.tools
from keras.preprocessing.image import ImageDataGenerator

egitim_datagen = ImageDataGenerator(
    rescale=1. / 255,   
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

egitim_generator = egitim_datagen.flow_from_directory(
        'drive/YapayZeka/Validation',
        target_size=(224, 244),
        color_mode="rgb",
        batch_size=70,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        'drive/YapayZeka/Train',
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=70,
        class_mode='categorical')

Found 1551 images belonging to 4 classes.
Found 3020 images belonging to 4 classes.


In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD, Adam, Adadelta
from keras.regularizers import l2
import keras.backend as K
import math
#-------------------NOT---------------HANGİSİNİ KULLANACAKSAN
#from tensorflow.keras.applications.xception import Xception
#from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Input


#-------------------NOT---------------
#model = HANGİSİNİKULLANDIYSAN(weights = 'ima...) 
model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
#EGİTİM ,VALİDATİON SAYISINI YAZMAYI EPOCHS SAYISINI DEĞİŞTİRMEYİ UNUTMAYIN.
model.fit_generator(egitim_generator,
                    validation_data=test_generator,
                    validation_steps=1551//70,   #   validation verisi/batchSize
                    steps_per_epoch=3020 //70,  #  egitim verisi/batchSize
                    epochs=4,
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/4
23/43 [===============>..............] - ETA: 46:35 - loss: 23.2250 - accuracy: 0.2785WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 172 batches). You may need to use the repeat() function when building your dataset.


KeyboardInterrupt: ignored